In [1]:
# Uncomment and run if you do not have these packages installed or are outdated
#!pip install python-louvain  --upgrade
#!pip install folium  --upgrade
#!pip install networkx --upgrade

# Graph Clustering

This example uses the frequent paths obtained from the twitter data to discover graph clusters

In [2]:
import community
import networkx as nx
from networkx.algorithms.community.centrality import girvan_newman
from networkx.algorithms.community.kernighan_lin import kernighan_lin_bisection

import folium

import warnings
warnings.filterwarnings('ignore')

def colorize(v):
    v = int(2**24 /(v+50)*15)
    return('#%s'%hex(v)[2:])

## Frequent Common locations from twitter

We will use the frequent paths from London (51.23, 51.8, -0.5, 0.25). You can also use "bcntwitter" (41.20, 41.6, 1.90, 2.40) and "paristwitter" (48.7, 49.05, 1.97, 2.68) and the networkx graph library. 

These paths have been computed as the frequent sequences of locations during a day for persons using twitter (colleted over a period of one year approx.).

First we will build a graph with all the frequent paths and we keep only the largest component of the graph

In [3]:
dpath = '../Data/'
datasets = {'LON': ('londontwitter.txt', (51.23, 51.8, -0.5, 0.25)),
            'BCN': ('bcntwitter.txt',  (41.20, 41.6, 1.90, 2.40)),
            'PAR': ('paristwitter.txt',(48.7, 49.05, 1.97, 2.68))
           }
fname, coord = datasets['LON']
rfile = open(dpath + fname, 'r')

gr = nx.Graph()

mymap = folium.Map(location=[(coord[0] + coord[1]) / 2.0, (coord[2] + coord[3]) / 2.0],
                   zoom_start=13, width=1000,
                   height=700, tiles='Stamen Toner')

for lines in rfile:
    vals = lines.replace('[', '').replace(']','').replace('\n','').replace('\'','').replace(' ','').split(',')
    for v1 in vals:
        for v2 in vals:
            if v1 != v2:
                gr.add_edge(v1,v2)               

c = max(nx.connected_components(gr), key=len) # Largest connected component
Gc = gr.subgraph(c).copy() 

maplines = []  
for ed in Gc.edges():
    x1, y1, _= ed[0].split('#')
    x2, y2, _= ed[1].split('#')
    maplines.append(folium.PolyLine(locations=[(float(x1), float(y1)), (float(x2), float(y2))], color='red', opacity=1.0, weight=2))

for ln in maplines:
     mymap.add_child(ln)

In [4]:
mymap

## Bipartition - Kernighan Lin

In [5]:
klpartition = kernighan_lin_bisection(Gc, max_iter=200)

mymap = folium.Map(location=[(coord[0] + coord[1]) / 2.0, (coord[2] + coord[3]) / 2.0], zoom_start=13, width=1000,
                   height=700, tiles='Stamen Toner')
for partition, c in zip(klpartition, ['#0000FF', '#00FF00']):
    for vert in partition:
        x1, y1, _= vert.split('#')
        folium.CircleMarker(location=[float(x1),float(y1)], radius=10,color=c, fill_color=c, fill=True).add_to(mymap)


In [6]:
mymap

## Communities - Girvan Newman

In [7]:
import itertools

k = 12
comp = girvan_newman(Gc)
limited = itertools.takewhile(lambda c: len(c) <= k, comp)
gnpartition = [communities for communities in limited][-1]

mymap = folium.Map(location=[(coord[0] + coord[1]) / 2.0, (coord[2] + coord[3]) / 2.0], zoom_start=13, width=1000,
                   height=700, tiles='Stamen Toner')
for i, partition in enumerate(gnpartition):
    c = colorize(i)
    for vert in partition:
        x1, y1, _= vert.split('#')
        folium.CircleMarker(location=[float(x1),float(y1)], radius=10,color=c, fill_color=c, fill=True).add_to(mymap)            

In [8]:
mymap

## Communities - Louvain

We can apply a community discovery algorithm from social networks to partition the graph. We will use the community python package http://perso.crans.org/aynaud/communities/index.html

In [9]:
lvpartition = community.best_partition(Gc)

mymap = folium.Map(location=[(coord[0] + coord[1]) / 2.0, (coord[2] + coord[3]) / 2.0], zoom_start=13, width=1000,
                   height=700, tiles='Stamen Toner')
for vert in lvpartition:
    x1, y1, _= vert.split('#')
    c = colorize(lvpartition[vert])
    folium.CircleMarker(location=[float(x1),float(y1)], radius=10,color=c, fill_color=c, fill=True).add_to(mymap)
    

In [10]:
mymap